In [2]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/politifact_raw.csv')
df.head()

,author,statement,source,date,target,article_url,src_url,highlight,article,references,categories,category_urls
0,Ciara O'Rourke,"“The airport in Salzburg, Austria, has a count...",Instagram posts,"October 30, 2023",false,https://www.politifact.com/factchecks/2023/oct...,https://www.politifact.com/personalities/insta...,This rumor stems from an ad. No such counter e...,A social media post poised to encourage people...,"['Instagram post', 'The Local', 'The Washingto...","['Transportation', 'Facebook Fact-checks', 'In...","['https://www.politifact.com/transportation/',..."
1,Ciara O'Rourke,Video shows Palestinians pretending to be corp...,Viral image,"October 30, 2023",false,https://www.politifact.com/factchecks/2023/oct...,https://www.politifact.com/personalities/viral...,This video is 10 years old and shows student p...,The Gaza Health Ministry has said the Palestin...,"['Instagram post', 'Instagram post', 'YouTube']","['Israel', 'Facebook Fact-checks', 'Viral image']","['https://www.politifact.com/israel/', 'https:..."
2,Loreben Tuquero,The life span of a wind tower generator lasts ...,Facebook posts,"October 30, 2023",false,https://www.politifact.com/factchecks/2023/oct...,https://www.politifact.com/personalities/faceb...,A study by energy industry experts showed that...,Let’s clear the air. Do wind turbine component...,"['Facebook post (archived)', 'X post', 'Facebo...","['Climate Change', 'Energy', 'Facebook Fact-ch...","['https://www.politifact.com/climate-change/',..."
3,Ciara O'Rourke,Matthew Perry died because of a COVID-19 vaccine.,Instagram posts,"October 30, 2023",false,https://www.politifact.com/factchecks/2023/oct...,https://www.politifact.com/personalities/insta...,"Actor Matthew Perry died Oct. 28, but his caus...","Actor Matthew Perry died Oct. 28, setting off ...","['Instagram post', 'Instagram post', 'Instagra...","['Facebook Fact-checks', 'Coronavirus', 'Insta...",['https://www.politifact.com/facebook-fact-che...
4,Jill Terreri,A discrepancy in the number of ballots and vot...,New York Citizens Audit,"October 29, 2023",false,https://www.politifact.com/factchecks/2023/oct...,https://www.politifact.com/personalities/new-y...,New York Citizens Audit is comparing certified...,The claim is startling: New York’s election re...,"['Email interview', 'Email interview', 'Email ...","['Elections', 'New York', 'New York Citizens A...","['https://www.politifact.com/elections/', 'htt..."


### `references` - list of string to string
- Manual stemmer
- Replace `NaN` with `none`
- Replace white space with `'_'` for BoW

In [ ]:
import ast
df['references'] = df['references'].apply(ast.literal_eval)

In [ ]:
res = df.explode('references')['references']
res = res.str.extract(r"(^[a-zA-Z0-9_ ]*$)", expand=False).fillna('none').apply(str.lower).apply(str.strip)
res[res.str.contains('disease control and prevention') == True] = 'centers for disease control and prevention'
res[res.str.contains('washington post') == True] = 'washington post'
res[res.str.contains('politifact') == True] = 'politifact'
res[res.str.contains('youtube') == True] = 'youtube'
res[res.str.contains('interview') == True] = 'interview'
res[res.str.contains('email') == True] = 'email'
res[res.str.contains('report') == True] = 'report'
res[res.str.contains('web') == True] = 'webpage'
res[res.str.contains('article') == True] = 'article'
res[res.str.contains('nbc') == True] = 'nbc'
res[res.str.contains('bbc') == True] = 'bbc'
res[res.str.contains('abc') == True] = 'abc'
res[res.str.contains('cbs') == True] = 'cbs'
res[res.str.contains('fox') == True] = 'fox'
res[res.str.contains('congress') == True] = 'congress'
res[(res.str.contains('tweet') == True) | (res.str.contains('twitter') == True)] = 'x post'
res[(res.str.contains('meta') == True) | (res.str.contains('facebook') == True)] = 'facebook post'
res[res.str.contains('insta') == True] = 'instagram post'
res[res.str.contains('time')] = 'new york times'
res[res.str.contains('linkedin')] = 'linkedin'
res = res.str.replace(' ', '_')

In [ ]:
ref = df.explode('references')
ref['references'] = res
df['references'] = ref[['references']].groupby(ref.index).agg({'references': lambda x: ','.join(x)})

### `categories` - list of string to string
- Replace `NaN` with `none`
- Replace white space with `'_'` for BoW

In [ ]:
df['categories'] = df['categories'].apply(str.lower).apply(ast.literal_eval)

In [ ]:
cat = df.explode('categories')['categories']
cat = cat.str.replace('-', ' ')
cat = cat.str.extract(r"(^[a-zA-Z0-9_ ]*$)", expand=False).fillna('none').apply(str.lower).apply(str.strip)
cat = cat.str.replace(' ', '_')

In [ ]:
temp = df.explode('categories')
temp['categories'] = cat
df['categories'] = temp[['categories']].groupby(temp.index).agg({'categories': lambda x: ','.join(x)})

In [ ]:
df.rename(columns={'statement':'headline'}).drop(columns=['article_url']).to_csv('../data/politifact_raw.csv', index=None)

In [3]:
pd.read_csv('../data/politifact_plus.csv')

,source,when/where,headline,target,speaker,documented_time,author_score,summaries,article,src_true,src_mostly_true,src_half_true,src_mostly_false,src_false,src_pants_on_fire
0,Instagram posts,"stated on October 28, 2023 in a screenshot sha...",“Haaretz investigation reveals discrepancies i...,false,Madison Czopek,"October 31, 2023",[ 5 3 16 54 473 152],"['Haaretz, an Israeli newspaper, said on X tha...",A viral Oct. 28 social media post claimed that...,5.0,3.0,16.0,54.0,480.0,157.0
1,Scott Walker,"stated on May 30, 2023 in Interview:",“Wisconsin has historically … and I think larg...,barely-true,Laura Schulte,"October 31, 2023",[26 45 39 41 44 11],['Although Wisconsin has voted for more Democr...,"In 2016, Wisconsin helped to swing the preside...",26.0,45.0,39.0,41.0,44.0,11.0
2,Instagram posts,"stated on October 27, 2023 in a post:","“The airport in Salzburg, Austria, has a count...",false,Ciara O'Rourke,"October 30, 2023",[ 5 3 16 54 473 152],[],A social media post poised to encourage people...,5.0,3.0,16.0,54.0,480.0,157.0
3,Viral image,"stated on October 27, 2023 in an Instagram post:",Video shows Palestinians pretending to be corp...,false,Ciara O'Rourke,"October 30, 2023",[ 4 13 35 53 745 336],['This video is 10 years old and shows student...,The Gaza Health Ministry has said the Palestin...,4.0,13.0,35.0,53.0,764.0,340.0
4,Facebook posts,"stated on September 25, 2023 in a Facebook post:",The life span of a wind tower generator lasts ...,false,Loreben Tuquero,"October 30, 2023",[ 24 50 108 247 1519 594],['A study by energy industry experts showed th...,Let’s clear the air. Do wind turbine component...,24.0,50.0,108.0,247.0,1532.0,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25650,Mike Huckabee,"stated on June 5, 2007 in Manchester, N.H.:","""I'll tell you what I can tell this country: I...",false,Angie Drobnic Holan,"August 7, 2007",[ 8 3 9 12 6 4],[],"Huckabee says there are ""probably plenty"" of p...",8.0,3.0,9.0,12.0,6.0,4.0
25651,Rudy Giuliani,"stated on July 30, 2007 in a speech in Laconia...","""Democrats are kind of falling over each other...",half-true,John Martin,"August 6, 2007",[ 8 7 10 9 12 7],[],It's not entirely clear from the context wheth...,8.0,7.0,10.0,9.0,12.0,7.0
25652,Bill Richardson,"stated on May 10, 2007 in a TV Ad.:","""I used tax cuts to help create over 80,000 jo...",mostly-true,ill Adair,"August 1, 2007",[2 8 3 1 2 1],[],Statistics from the Bureau of Labor Statistics...,2.0,8.0,3.0,1.0,2.0,1.0
25653,Hillary Clinton,"stated on July 22, 2007 in Washington, DC:","""Corporate profits are up, CEO pay is up, but ...",true,ill Adair,"August 1, 2007",[72 76 70 43 31 9],[],Corporate profits have been rising. The Commer...,72.0,76.0,70.0,43.0,31.0,9.0
